In [28]:
! git clone https://github.com/tonythomasndm/CSE508_Winter2024_A2_Dataset.git

fatal: destination path 'CSE508_Winter2024_A2_Dataset' already exists and is not an empty directory.


In [29]:
#Preprocessing the dataset

import pandas as pd
df= pd.read_csv('CSE508_Winter2024_A2_Dataset/A2_Data.csv',header=None)
df.info()
df.rename(columns={0: 'Review Id',1:'Image',2:'Review Text'}, inplace=True)
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
df.info()
org_df=df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       1000 non-null   float64
 1   1       1001 non-null   object 
 2   2       1000 non-null   object 
dtypes: float64(1), object(2)
memory usage: 23.6+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Review Id    999 non-null    float64
 1   Image        999 non-null    object 
 2   Review Text  999 non-null    object 
dtypes: float64(1), object(2)
memory usage: 23.5+ KB


In [30]:
#Preprocessing the list of images into individual images only dataset with repeated information
#and removing those rows whose links don't work
def fnkk(imgArray):
    if ',' not in imgArray:
        arr=[]
        arr.append(imgArray[2:-2])
        return arr
    else:
        imgArr=imgArray[1:-1]
        lis=imgArr.split(',')
        lt=[]
        for i in lis:
            i=i.strip()
            i=i[1:-1]
            lt.append(i)
        lis=lt
        return lis

new_rows = {
    'Review Id':[],
    'Image':[],
    'Review Text':[]
}
for index, row in df.iterrows():
    res = fnkk(row['Image'])
    if(row['Image'])=="[]":
        continue
    pid = row['Review Id']
    rt = row['Review Text']
    for img_link in res:
        new_rows['Review Id'].append(pid)
        new_rows['Image'].append(img_link)
        new_rows['Review Text'].append(rt)

# Creating new DataFrame
new_df = pd.DataFrame(new_rows)
print(new_df.info())
new_df.to_csv('output_no_list_of_images_only_one_image_per_row.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1647 entries, 0 to 1646
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Review Id    1647 non-null   float64
 1   Image        1647 non-null   object 
 2   Review Text  1647 non-null   object 
dtypes: float64(1), object(2)
memory usage: 38.7+ KB
None


In [31]:
#Image Extraction - Extracting the image features using Inception
#Preprocessing the images and storing them 
#For each Image - it gave an array of dimension (5,5,2048)
#Preprocessed and extrcated features of 1639 working images
from keras.preprocessing import image
from keras.models import Model
from keras.applications.inception_v3 import InceptionV3, preprocess_input
import cv2
import numpy as np
import random
import requests
from PIL import Image
from io import BytesIO

df = new_df

ls=[]

def preprocess_image(url):
    try:
        response = requests.get(url)
        img = Image.open(BytesIO(response.content)).convert('RGB')
        img = img.resize((224, 224))  # Resize the image
        img = np.array(img) 
        if random.random() > 0.5:
            img = np.fliplr(img)
        alpha = 1.0 + random.uniform(-0.3, 0.3) 
        beta = random.randint(-50, 50)
        img = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)
        ls.append(1)
        return img
    except Exception as e:
        print(f"Error processing image at {url}: {e}")
        ls.append(0)
        return None

def extract_image_features(images):
    base_model = InceptionV3(weights='imagenet', include_top=False)
    model = Model(inputs=base_model.input, outputs=base_model.get_layer('mixed10').output)

    features = []
    for img_url in images:
        img = preprocess_image(img_url)
        if img is not None:
            img = np.expand_dims(img, axis=0)
            img = preprocess_input(img)
            feature = model.predict(img)[0]  # Take the first element of the predictions array
            features.append(feature)
    return np.array(features)

def normalize_image_features(image_features):
    normalized_features = (image_features - np.mean(image_features)) / np.std(image_features)
    return normalized_features

# Extract image features
image_urls = df['Image'].values
image_features = extract_image_features(image_urls)

normalized_image_features=[]
# Normalize image features
for i in image_features:
    normalfeatures=normalize_image_features(i)
    normalized_image_features.append(normalfeatures)

#normalized_image_features = (image_features - np.mean(image_features, axis=0)) / np.std(image_features, axis=0)

import pickle
# Save image features using pickle
with open('image_features.pkl', 'wb') as f:
    pickle.dump(normalized_image_features, f)

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 

In [32]:
df.info()
org_df.info()

#Here df - represents the df with 1647 images-row df
#the org_df represents the df with 999 rows after preprocessing the dataset - removing the null rows


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1647 entries, 0 to 1646
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Review Id    1647 non-null   float64
 1   Image        1647 non-null   object 
 2   Review Text  1647 non-null   object 
dtypes: float64(1), object(2)
memory usage: 38.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Review Id    999 non-null    float64
 1   Image        999 non-null    object 
 2   Review Text  999 non-null    object 
dtypes: float64(1), object(2)
memory usage: 23.5+ KB


In [33]:
#Finding out the errorneous review Ids and creating a new dataframe that stores only those
#image links that are working
#Our dataset gets reduced from 1647 to 1639 rows and found 6 errorneous review ids
new_df={
    'Review Id':[],
    'Image':[],
    'Review Text':[]
}

remove_review_id=[]
print("Errorness Review Ids")
for i in range(len(ls)):
    if ls[i] == 1:
        new_df['Review Id'].append(df.loc[i, 'Review Id'])
        new_df['Image'].append(df.loc[i, 'Image'])
        new_df['Review Text'].append(df.loc[i, 'Review Text'])
    else:
        remove_review_id.append(df.loc[i, 'Review Id'])
        print(df.loc[i, 'Review Id'])

new_df = pd.DataFrame(new_df)
new_df.info()

Errorness Review Ids
2235.0
2235.0
3317.0
3317.0
2912.0
2265.0
2088.0
3474.0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1639 entries, 0 to 1638
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Review Id    1639 non-null   float64
 1   Image        1639 non-null   object 
 2   Review Text  1639 non-null   object 
dtypes: float64(1), object(2)
memory usage: 38.5+ KB


In [34]:
org_df.info()
df.info()
new_df.info()

#org_df - presents the orginal datafarme with 999 non-null rows
#df - presents the expanded dataframe storing unique image links
#new_df - the similar expanded dataframe but removed the errorneous review ids 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Review Id    999 non-null    float64
 1   Image        999 non-null    object 
 2   Review Text  999 non-null    object 
dtypes: float64(1), object(2)
memory usage: 23.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1647 entries, 0 to 1646
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Review Id    1647 non-null   float64
 1   Image        1647 non-null   object 
 2   Review Text  1647 non-null   object 
dtypes: float64(1), object(2)
memory usage: 38.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1639 entries, 0 to 1638
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Review Id    1639 non-null   float64
 1   Image        1639 non-null 

In [35]:
#Putting the normalized image features into the dataframe in the list wise order as it is in the image column and then saving the file using pickle module as the final file 

import pickle

with open('image_features.pkl', 'rb') as f:
    normalized_image_features = pickle.load(f)

normalized_image_features_col=[]
index=0
for index, row in org_df.iterrows():
    res = fnkk(row['Image'])
    if(row['Image'])=="[]":
        continue
    lt=[]
    for img_link in res:
        if ls[index]==1:
            lt.append(normalized_image_features[index])
        else:
            continue
        index+=1
    normalized_image_features_col.append(lt)
  
print(len(normalized_image_features_col))

df=org_df
df['Normalized Image Features']=normalized_image_features_col

df.info()
#the erroenorus review ids(image links not working) are still not removed 
# but we have a stored list of errorneous review ids

999
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Review Id                  999 non-null    float64
 1   Image                      999 non-null    object 
 2   Review Text                999 non-null    object 
 3   Normalized Image Features  999 non-null    object 
dtypes: float64(1), object(3)
memory usage: 31.3+ KB


In [36]:
for index, row in df.iterrows():
    if row['Review Id'] in remove_review_id:
        df.drop(index, inplace=True)

df.reset_index(drop=True, inplace=True)


In [37]:
import pandas as pd
import numpy as np
import math
import re
import nltk
import pickle
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

def preprocess_text(text):
    text = text.lower()
    tokens = word_tokenize(text)
    # Removing punctuation and non-alphabetic characters
    tokens = [word for word in tokens if word.isalpha()]
    stop_words = set(stopwords.words("english"))
    tokens = [word for word in tokens if word not in stop_words]
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return tokens

df['Cleaned Review'] = df['Review Text'].apply(preprocess_text)

# Calculated term frequency (TF)
def calculate_tf(text):
    tf_dict = {}
    total_words = len(text)
    for word in text:
        tf_dict[word] = tf_dict.get(word, 0) + 1 / total_words
    return tf_dict

df['TF'] = df['Cleaned Review'].apply(calculate_tf)

# Calculated inverse document frequency (IDF)
def calculate_idf(documents):
    idf_dict = {}
    total_documents = len(documents)
    for document in documents:
        for word in set(document):
            idf_dict[word] = idf_dict.get(word, 0) + 1
    for word, val in idf_dict.items():
        idf_dict[word] = math.log10(total_documents / (val + 1))  # Add 1 to avoid division by zero
    return idf_dict

idf_dict = calculate_idf(df['Cleaned Review'])

# Combined TF and IDF to calculate TF-IDF vectors
def calculate_tfidf_vector(tf_dict, idf_dict):
    tfidf_vector = [tf_dict.get(word, 0) * idf_dict.get(word, 0) for word in idf_dict.keys()]
    return tfidf_vector

df['TF-IDF Vector'] = df.apply(lambda row: calculate_tfidf_vector(row['TF'], idf_dict), axis=1)

# Saved the IDF dictionary using pickle
with open('idf_dict.pkl', 'wb') as f:
    pickle.dump(idf_dict, f)

# Save the TF-IDF vectors using pickle
with open('tfidf_vectors.pkl', 'wb') as f:
    pickle.dump(df['TF-IDF Vector'], f)

for i in df['TF-IDF Vector']:
    i=np.array(i)
    print(i.shape)

(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)
(4122,)


In [38]:
# Save the updated DataFrame to a CSV file
df.to_csv("FinalFile.csv", index=False)
new_df.to_csv("NewDfFinalFile.csv", index=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 993 entries, 0 to 992
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Review Id                  993 non-null    float64
 1   Image                      993 non-null    object 
 2   Review Text                993 non-null    object 
 3   Normalized Image Features  993 non-null    object 
 4   Cleaned Review             993 non-null    object 
 5   TF                         993 non-null    object 
 6   TF-IDF Vector              993 non-null    object 
dtypes: float64(1), object(6)
memory usage: 54.4+ KB


In [39]:
#Start running from here
import pickle
import pandas as pd

with open('idf_dict.pkl', 'rb') as f:
    idf_dict = pickle.load(f)
with open('image_features.pkl', 'rb') as f:
    normalized_image_features = pickle.load(f)

df=pd.read_csv('FinalFile.csv')
new_df=pd.read_csv('NewDfFinalFile.csv')
df.info()
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 993 entries, 0 to 992
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Review Id                  993 non-null    float64
 1   Image                      993 non-null    object 
 2   Review Text                993 non-null    object 
 3   Normalized Image Features  993 non-null    object 
 4   Cleaned Review             993 non-null    object 
 5   TF                         993 non-null    object 
 6   TF-IDF Vector              993 non-null    object 
dtypes: float64(1), object(6)
memory usage: 54.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1639 entries, 0 to 1638
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Review Id    1639 non-null   float64
 1   Image        1639 non-null   object 
 2   Review Text  1639 non-null   object 
dtypes: float64(1), obj

In [40]:
"Getting Input"
print("Enter User Review")
input_review_text=input()
print(input_review_text)
print("Enter Image Link")
input_image=input()
print(input_image)

Enter User Review
Loving these vintage springs on my vintage strat. They have a good tension and great stability. If you are floating your bridge and want the most out of your springs than these are the way to go.
Enter Image Link
https://images-na.ssl-images-amazon.com/images/I/81q5+IxFVUL._SY88.jpg


In [41]:
from keras.preprocessing import image
from keras.applications.inception_v3 import InceptionV3, preprocess_input
import cv2
import numpy as np
import requests
from PIL import Image
from io import BytesIO

# def preprocess_image(url):
#     try:
#         response = requests.get(url)
#         img = Image.open(BytesIO(response.content)).convert('RGB')
#         img = img.resize((224, 224))  # Resize the image
#         img = np.array(img) 
#         if random.random() > 0.5:
#             img = np.fliplr(img)
#         alpha = 1.0 + random.uniform(-0.3, 0.3) 
#         beta = random.randint(-50, 50)
#         img = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)
#         ls.append(1)
#         return img
#     except Exception as e:
#         print(f"Error processing image at {url}: {e}")
#         ls.append(0)
#         return None

# def extract_image_features(images):
#     base_model = InceptionV3(weights='imagenet', include_top=False)
#     model = Model(inputs=base_model.input, outputs=base_model.get_layer('mixed10').output)

#     features = []
#     for img_url in images:
#         img = preprocess_image(img_url)
#         if img is not None:
#             img = np.expand_dims(img, axis=0)
#             img = preprocess_input(img)
#             feature = model.predict(img)[0]  # Take the first element of the predictions array
#             features.append(feature)
#     return np.array(features)

def normalize_image_features(image_features):
    normalized_features = (image_features - np.mean(image_features)) / np.std(image_features)
    return normalized_features

# URL of the input image
input_image_url = input_image

# Extract features for the input image
input_image_features = extract_image_features([input_image_url])[0]
print(input_image_features.shape)
# Preprocess and normalize the input image features
normalized_input_image_features = normalize_image_features(input_image_features)

# Save the normalized input image features using pickle if needed
with open('normalized_input_image_features.pkl', 'wb') as f:
    pickle.dump(normalized_input_image_features, f)

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
(5, 5, 2048)


In [42]:
print(normalized_input_image_features)
normalized_input_image_features.shape

[[[ 0.6244217   4.6278334  -0.6109296  ... -0.6109296  -0.33786377
    1.5066687 ]
  [ 1.6503359   1.2068639  -0.6109296  ... -0.6109296  -0.59912026
    1.5503616 ]
  [ 1.0957708   1.8557917   0.5030178  ... -0.6109296  -0.24112146
    0.6402442 ]
  [ 0.72469836 -0.6109296  -0.6109296  ... -0.6109296  -0.15441285
   -0.25370273]
  [-0.6109296   0.39386076 -0.6109296  ... -0.6109296   0.32919726
   -0.6109296 ]]

 [[ 0.84427935  1.0514433  -0.6109296  ... -0.6109296  -0.18322481
    0.7163834 ]
  [-0.34625167  0.177018   -0.21378757 ... -0.6109296  -0.4237584
    0.7636776 ]
  [-0.6109296  -0.6109296   0.46369776 ... -0.6109296  -0.279065
    0.08994146]
  [-0.6109296  -0.5337417  -0.09434924 ... -0.6109296  -0.26842767
   -0.5818557 ]
  [-0.6109296   1.2280532   1.3027014  ... -0.6109296   0.04977177
   -0.6109296 ]]

 [[-0.2839179   0.6085616  -0.6109296  ... -0.6109296  -0.08609613
   -0.46804577]
  [-0.6109296   1.1005     -0.6109296  ... -0.6109296  -0.08705458
   -0.6109296 ]
  [

(5, 5, 2048)

In [43]:
#flattening the from 3d array to 1d array
import numpy as np

flattened_normalized_image_features=[]
for i in normalized_image_features:
    sd=np.array(i)
    entry_array = np.array(sd)
    entry_array= entry_array.reshape(-1)
    flattened_normalized_image_features.append(entry_array)


new_df['Normalized Image Features'] = flattened_normalized_image_features

flattened_normalized_image_features=np.array(flattened_normalized_image_features)
print(flattened_normalized_image_features.shape)


(1639, 51200)


In [44]:
new_df.info() #expanded dataframe
df.info() #993 rows dataframe

#but we have a issue the normalized image features in df is 3d arry - it is not flattened hence

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1639 entries, 0 to 1638
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Review Id                  1639 non-null   float64
 1   Image                      1639 non-null   object 
 2   Review Text                1639 non-null   object 
 3   Normalized Image Features  1639 non-null   object 
dtypes: float64(1), object(3)
memory usage: 51.3+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 993 entries, 0 to 992
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Review Id                  993 non-null    float64
 1   Image                      993 non-null    object 
 2   Review Text                993 non-null    object 
 3   Normalized Image Features  993 non-null    object 
 4   Cleaned Review             993 non-null    object 
 5   TF  

In [45]:
#Editing and adding the flattened array dataset to df['Normalized Image Features]

normalized_image_features_col=[]
inde=0
for index, row in df.iterrows():
    res = fnkk(row['Image'])
    if(row['Image'])=="[]":
        continue
    lt=[]
    for img_link in res:
        lt.append(new_df.loc[inde,'Normalized Image Features'])
        inde+=1
    normalized_image_features_col.append(lt)

print(len(normalized_image_features_col))

df['Normalized Image Features']=normalized_image_features_col

993


In [46]:
import pickle
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords

# Load IDF dictionary from pickle file
with open('idf_dict.pkl', 'rb') as f:
    idf_dict = pickle.load(f)

def preprocess_text(text):
    text = text.lower()
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word.isalpha()]
    stop_words = set(stopwords.words("english"))
    tokens = [word for word in tokens if word not in stop_words]
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return tokens

# Calculating TF-IDF vector for input text
def calculate_tfidf_vector(input_text, idf_dict):
    preprocessed_text = preprocess_text(input_text)
    tf_dict = {}
    total_words = len(preprocessed_text)
    for word in preprocessed_text:
        tf_dict[word] = tf_dict.get(word, 0) + 1 / total_words
    tfidf_vector = [tf_dict.get(word, 0) * idf_dict.get(word, 0) for word in idf_dict.keys()]
    return tfidf_vector

# Calculating TF-IDF vector for input review text
tfidf_vector_input = calculate_tfidf_vector(input_review_text, idf_dict)

print("TF-IDF Vector for Input Review Text:")
print(len(tfidf_vector_input))


TF-IDF Vector for Input Review Text:
4122


In [50]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pickle

# Load TF-IDF vectors from pickle file
with open('tfidf_vectors.pkl', 'rb') as f:
    tfidf_vectors = pickle.load(f)

df['TF-IDF Vector'] = tfidf_vectors

def cosine_similarity_text(tfidf1, tfidf2):
    tfidf1_array=np.array(tfidf1)
    tfidf2_array=np.array(tfidf2)
    return cosine_similarity(tfidf1_array.reshape(1, -1), tfidf2_array.reshape(1, -1))[0][0]

def cosine_similarity_image(list_of_image_features, input_image_features):
    similarties=[]
    input_image_features=np.array(input_image_features)
    for i in list_of_image_features:
        similarties.append(cosine_similarity(i.reshape(1, -1),input_image_features.reshape(1, -1))[0][0])
    return np.mean(similarties)

n=int(input("Enter the number of ranks required:"))
print(n)

cosine_similarity_text_all = []
for value in df['TF-IDF Vector']:
    val = cosine_similarity_text(value, tfidf_vector_input)
    cosine_similarity_text_all.append(val)

df['Cosine Similarity Text'] = cosine_similarity_text_all
top_text=df.nlargest(n, 'Cosine Similarity Text').index

cosine_similarity_image_all = []
for value in df['Normalized Image Features']:
    val = cosine_similarity_image(value, normalized_input_image_features)
    cosine_similarity_image_all.append(val)

df['Cosine Similarity Image'] = cosine_similarity_image_all
top_image=df.nlargest(n, 'Cosine Similarity Image').index

df['Composite Similarity'] = (df['Cosine Similarity Text'] + df['Cosine Similarity Image']) / 2.0


top_composite_score_indices = df.nlargest(n, 'Composite Similarity').index

# Print results
print("Text Based Retrieval - ranked based on text cosine similarity")
for idx in top_text:
    review_id = df.loc[idx, 'Review Id']
    review_text = df.loc[idx, 'Review Text']
    image_urls = df.loc[idx, 'Image']
    text_cosine_similarity = df.loc[idx, 'Cosine Similarity Text']
    average_image_similarity = df.loc[idx, 'Cosine Similarity Image']
    composite_score = (text_cosine_similarity+average_image_similarity)/2.0
    print(f"Image URLs: {image_urls}")
    print(f"Review ID: {review_id}")
    print(f"Review Text: {review_text}")
    print(f"Text-Cosine Similarity: {text_cosine_similarity}")
    print(f"Average Image Similarity: {average_image_similarity}")
    print(f"Composite Similarity: {composite_score}")
    print()
print("\n\n\n")
print("Image Based Retrieval - ranked based on image cosine similarity")
for idx in top_image:
    review_id = df.loc[idx, 'Review Id']
    review_text = df.loc[idx, 'Review Text']
    image_urls = df.loc[idx, 'Image']
    text_cosine_similarity = df.loc[idx, 'Cosine Similarity Text']
    average_image_similarity = df.loc[idx, 'Cosine Similarity Image']
    composite_score = (text_cosine_similarity+average_image_similarity)/2.0
    print(f"Image URLs: {image_urls}")
    print(f"Review ID: {review_id}")
    print(f"Review Text: {review_text}")
    print(f"Text-Cosine Similarity: {text_cosine_similarity}")
    print(f"Average Image Similarity: {average_image_similarity}")
    print(f"Composite Similarity: {composite_score}")
    print()
print("\n\n\n")



3
Text Based Retrieval - ranked based on text cosine similarity
Image URLs: ['https://images-na.ssl-images-amazon.com/images/I/81q5+IxFVUL._SY88.jpg']
Review ID: 3452.0
Review Text: Loving these vintage springs on my vintage strat. They have a good tension and great stability. If you are floating your bridge and want the most out of your springs than these are the way to go.
Text-Cosine Similarity: 0.9999999999999999
Average Image Similarity: 0.5282899141311646
Composite Similarity: 0.7641449570655823

Image URLs: ['https://images-na.ssl-images-amazon.com/images/I/81Z1d7HaBfL._SY88.jpg']
Review ID: 2220.0
Review Text: Nice solid springs and defeinitely more silent. Easy installation and the black looks cool.

Pictured with some old uninstalled springs next to them.
Text-Cosine Similarity: 0.34533063572168526
Average Image Similarity: 0.08322581648826599
Composite Similarity: 0.21427822610497563

Image URLs: ['https://images-na.ssl-images-amazon.com/images/I/61rFj+KtBRL._SY88.jpg']
Revi

In [48]:
#Save the results

import pickle

# Create a dictionary to store the results
results = {
    'top_text_indices': top_text,
    'top_image_indices': top_image,
    'top_composite_indices': top_composite_score_indices
}

# Save the results dictionary using pickle
with open('retrieval_results.pkl', 'wb') as f:
    pickle.dump(results, f)


In [49]:
import pickle

# Load the results dictionary from the pickled file
with open('retrieval_results.pkl', 'rb') as f:
    loaded_results = pickle.load(f)

# Extract the top text indices, top image indices, and top composite indices
top_text_indices = loaded_results['top_text_indices']
top_image_indices = loaded_results['top_image_indices']
top_composite_indices = loaded_results['top_composite_indices']

# Sort the indices based on the composite score
top_text_indices_sorted = sorted(top_text_indices, key=lambda idx: (df.loc[idx, 'Cosine Similarity Text']+df.loc[idx, 'Cosine Similarity Image'])/2.0, reverse=True)
top_image_indices_sorted = sorted(top_image_indices, key=lambda idx: (df.loc[idx, 'Cosine Similarity Text']+df.loc[idx, 'Cosine Similarity Image'])/2.0, reverse=True)
top_composite_indices_sorted = sorted(top_composite_indices, key=lambda idx: df.loc[idx, 'Composite Similarity'], reverse=True)

# Print results
print("Text Based Retrieval - ranked based on composite score")
for idx in top_text_indices_sorted:
    review_id = df.loc[idx, 'Review Id']
    review_text = df.loc[idx, 'Review Text']
    image_urls = df.loc[idx, 'Image']
    text_cosine_similarity = df.loc[idx, 'Cosine Similarity Text']
    average_image_similarity = df.loc[idx, 'Cosine Similarity Image']
    composite_score = (text_cosine_similarity+average_image_similarity)/2.0
    print(f"Image URLs: {image_urls}")
    print(f"Review ID: {review_id}")
    print(f"Review Text: {review_text}")
    print(f"Text-Cosine Similarity: {text_cosine_similarity}")
    print(f"Average Image Similarity: {average_image_similarity}")
    print(f"Composite Similarity: {composite_score}")
    print()
print("\n\n\n")
print("Image Based Retrieval -ranked based on composite score")
for idx in top_image_indices_sorted:
    review_id = df.loc[idx, 'Review Id']
    review_text = df.loc[idx, 'Review Text']
    image_urls = df.loc[idx, 'Image']
    text_cosine_similarity = df.loc[idx, 'Cosine Similarity Text']
    average_image_similarity = df.loc[idx, 'Cosine Similarity Image']
    composite_score = (text_cosine_similarity+average_image_similarity)/2.0
    print(f"Image URLs: {image_urls}")
    print(f"Review ID: {review_id}")
    print(f"Review Text: {review_text}")
    print(f"Text-Cosine Similarity: {text_cosine_similarity}")
    print(f"Average Image Similarity: {average_image_similarity}")
    print(f"Composite Similarity: {composite_score}")
    print()
print("\n\n\n")
print("Composite Score Retrieval - Combined Retrieval -ranked based on composite score")
for idx in top_composite_indices_sorted:
    review_id = df.loc[idx, 'Review Id']
    review_text = df.loc[idx, 'Review Text']
    image_urls = df.loc[idx, 'Image']
    text_cosine_similarity = df.loc[idx, 'Cosine Similarity Text']
    average_image_similarity = df.loc[idx, 'Cosine Similarity Image']
    composite_score = (text_cosine_similarity+average_image_similarity)/2.0
    print(f"Image URLs: {image_urls}")
    print(f"Review ID: {review_id}")
    print(f"Review Text: {review_text}")
    print(f"Text-Cosine Similarity: {text_cosine_similarity}")
    print(f"Average Image Similarity: {average_image_similarity}")
    print(f"Composite Similarity: {composite_score}")
    print()

Text Based Retrieval - ranked based on composite score
Image URLs: ['https://images-na.ssl-images-amazon.com/images/I/81q5+IxFVUL._SY88.jpg']
Review ID: 3452.0
Review Text: Loving these vintage springs on my vintage strat. They have a good tension and great stability. If you are floating your bridge and want the most out of your springs than these are the way to go.
Text-Cosine Similarity: 0.9999999999999999
Average Image Similarity: 0.5282899141311646
Composite Similarity: 0.7641449570655823

Image URLs: ['https://images-na.ssl-images-amazon.com/images/I/81Z1d7HaBfL._SY88.jpg']
Review ID: 2220.0
Review Text: Nice solid springs and defeinitely more silent. Easy installation and the black looks cool.

Pictured with some old uninstalled springs next to them.
Text-Cosine Similarity: 0.34533063572168526
Average Image Similarity: 0.08322581648826599
Composite Similarity: 0.21427822610497563

Image URLs: ['https://images-na.ssl-images-amazon.com/images/I/61rFj+KtBRL._SY88.jpg']
Review ID: 16

5a) Images based retreival- ranked based on average image average cosine score
Text based retrieval - ranked based on text cosine similarity
Images based retreival- ranked based on average image average cosine score -sorted top 3 on composite score
Text based retrieval - ranked based on text cosine similarity -sorted top 3 on composite score
combined retrieval- - sorted top 3 on composite score


5b) The Text Retrieval works the best for me , beacuse the text preprocessing is strong and has performed the stemming and lemmitaization, therfore the size of tf-idf vector is also 4122 which is small again
The Image Retrieval doesn't work well, again beacuse of preprocessing , as I have used random flips and geometric orientation change brightness and exposure change hence, it is not able to give closer cosine similarity, although it retrieves the best and is working almost fine, the reason being its cosine similarity is weak as we have multiple images of images to process that's why, moreover it considers the image features, but not the semantic meaning as considered in the text-based retrieval

5c) Potential Challenges faced
- the dataset had list of urls instaed of a single url each row- hence it was difficult to preprocess_image
- it was new to work with the inceptionv3 , a new cnn trained Model
- had to find the hperparameters for the preprocessing which was learnt through trial and error
- some imagesUrls were not working - had to find the errorneous review ids and remove them for proper processing
- Running a pretrained cnn model took a lot of time to preprocess the images
- There were many dimesnions , had to understand each output and understand its shape and reshape it according to the output

Potential Improvements include
- Including the Auditory Perception as well would be a good idea for a multimodal system 
- User Feedback should be integrated - using relevance feedback
- data should be more augmented as well, right now , we have a very small dataset
- User centric metrics can be used to capture the user-satisfaction with the model 
- We can also finetune the cnn model better according to the dataset, for more dynamic results
- Ensemble Methods can be used to combine multiple retrieval methods.Techniques such as stacking or blending can leverage the strengths of individual models to improve overall retrieval performance.
- We can enhance the system's semantic understanding capabilities by incorporating contextual information and domain-specific knowledge graphs. This can improve the system's ability to capture the semantics of user queries and content, leading to more relevant retrieval results.
- We can also develop hybrid retrieval approaches that integrate information from both textual and visual modalities at different stages of the retrieval process. Fusion techniques, such as late fusion (combining results at the retrieval stage) or early fusion (combining features from both modalities at the feature level), can leverage the complementary nature of text and images.
- We can also enhance the system's semantic understanding capabilities by incorporating contextual information and domain-specific knowledge graphs. This can improve the system's ability to capture the semantics of user queries and content, leading to more relevant retrieval results.